In [1]:
# video:   https://www.bilibili.com/video/BV1eR4y1H7EF?share_source=copy_web&vd_source=2591a9c2e8e224d8bd8e7145ff62deae

In [2]:
from nltk import data

In [3]:
data.path.append(r'../nltk_data')

In [4]:
import pandas as pd

In [5]:
import matplotlib.pyplot as plt

In [6]:
%matplotlib inline

In [7]:
import re

In [8]:
import string

In [9]:
from nltk import word_tokenize

In [10]:
from nltk.stem import WordNetLemmatizer

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
from collections import Counter

In [13]:
from pylab import *

In [14]:
import nltk

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
data_patio_lawn_garden = pd.read_json('data/reviews_Patio_Lawn_and_Garden.json', lines =True)
data_patio_lawn_garden[['reviewText', 'overall']].head()

,reviewText,overall
0,This was a gift for my other husband. He's ma...,5
1,"My husband rarely asks for anything specific, ...",5
2,This guy knows his stuff. I love the seriously...,5
3,"THIS CAN BE FOUND AT: [...] FOR $19.99, also a...",5
4,Quite good particularly for inspiration (DVD g...,5


In [17]:
lemmatizer = WordNetLemmatizer()
data_patio_lawn_garden['cleaned_review_text'] = data_patio_lawn_garden['reviewText'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))

In [18]:
data_patio_lawn_garden[['cleaned_review_text', 'reviewText', 'overall']].head()

,cleaned_review_text,reviewText,overall
0,this wa a gift for my other husband he s makin...,This was a gift for my other husband. He's ma...,5
1,my husband rarely asks for anything specific b...,"My husband rarely asks for anything specific, ...",5
2,this guy know his stuff i love the seriously s...,This guy knows his stuff. I love the seriously...,5
3,this can be found at for 19 99 also an issue i...,"THIS CAN BE FOUND AT: [...] FOR $19.99, also a...",5
4,quite good particularly for inspiration dvd gi...,Quite good particularly for inspiration (DVD g...,5


In [19]:
tfidf_model = TfidfVectorizer(max_features=500)
tfidf_df = pd.DataFrame(tfidf_model.fit_transform(data_patio_lawn_garden['cleaned_review_text']).todense())
tfidf_df.columns = sorted(tfidf_model.vocabulary_)
tfidf_df.head()

,10,12,15,20,30,34,able,about,actually,add,...,work,worked,working,worth,would,yard,year,yet,you,your
0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.095185,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.076392,0.0
1,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.18812,0.0,0.0,0.205174,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.185652,0.0,0.0,0.0,0.0,0.207186,0.0
4,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [20]:
data_patio_lawn_garden['target']=data_patio_lawn_garden['overall'].apply(lambda x : 0 if x<=4 else 1)

In [21]:
data_patio_lawn_garden['target'].value_counts()

1    561123
0    432367
Name: target, dtype: int64

In [22]:
def clf_model(model_type, X_train, y):
    model = model_type.fit(X_train,y)
    predicted_labels = model.predict(tfidf_df)
    return predicted_labels

In [23]:
from xgboost import XGBClassifier

In [24]:
xgb_clf=XGBClassifier(n_estimators=20,
                     learning_rate=0.03,
                     max_depth=5,
                     subsample=0.6,
                     colsample_bytree=0.6,
                     reg_alpha=10,
                     seed=42)

In [25]:
data_patio_lawn_garden['predicted_labels_xgbc'] = clf_model(xgb_clf, tfidf_df, data_patio_lawn_garden['target'])                                                        

[06:52:48] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [26]:
pd.crosstab(data_patio_lawn_garden['target'], data_patio_lawn_garden['predicted_labels_xgbc'])

predicted_labels_xgbc,0,1
target,,
0,254928,177439
1,119661,441462


In [27]:
def reg_model(model_type, X_train, y):
    model = model_type.fit(X_train, y)
    predicted_values = model.predict(tfidf_df)
    return predicted_values

In [28]:
from xgboost import XGBRegressor
xgbr = XGBRegressor(n_estimators=20,
                     learning_rate=0.03,
                     max_depth=5,
                     subsample=0.6,
                     colsample_bytree=0.6,
                     reg_alpha=10,
                     seed=42)

In [29]:
data_patio_lawn_garden['predicted_values_xgbr'] = reg_model(xgbr, tfidf_df, data_patio_lawn_garden['overall'])

In [30]:
data_patio_lawn_garden[['overall', 'predicted_values_xgbr']].head(2)

,overall,predicted_values_xgbr
0,5,2.398897
1,5,2.123649
